# Classification

### Preprocessing

In [1]:
import pandas as pd

df = pd.read_csv('movie_dataset_classification.csv',index_col=0)
df.dropna(subset=['Plot'], inplace=True)
df.drop_duplicates(subset=['Title','Plot'], inplace=True)
#removing low frequency genres
counts = df.Genre.value_counts()
counts = list(counts[counts < 150].index)
df = df[~df['Genre'].isin(counts)]
df.head(200)

,Release Year,Title,Director,Cast,Plot,Genre
0,1903.0,The Great Train Robbery,Edwin S. Porter,NaN,The film opens with two bandits breaking into ...,western
1,1904.0,The Suburbanite,Wallace McCutcheon,NaN,The film is about a family who move to the sub...,comedy
2,1906.0,From Leadville to Aspen: A Hold-Up in the Rockies,Francis J. Marion and Wallace McCutcheon,NaN,The film features a train traveling through th...,action
3,1907.0,Daniel Boone,Wallace McCutcheon and Ediwin S. Porter,"William Craven, Florence Lawrence",Boone's daughter befriends an Indian maiden as...,historical
4,1907.0,How Brown Saw the Baseball Game,Unknown,Unknown,Before heading out to a baseball game at a nea...,comedy
...,...,...,...,...,...,...
196,1918.0,Set Free,Tod Browning,"Edith Roberts, Harold Goodwin",After discovering that her grandmother was a g...,drama
197,1918.0,Shoulder Arms,Charles Chaplin,"Charlie Chaplin, Sydney Chaplin","Charlie is in boot camp in the ""awkward squad....",comedy
198,1918.0,Social Briars,Henry King,"Mary Miles Minter, Allan Forrest","As described in a film magazine,[3] Iris Lee (...",drama
199,1918.0,The Squaw Man,Cecil B. DeMille,"Elliott Dexter, Ann Little","As described in a film magazine,[3] Jim Wynneg...",western


In [29]:
df.Genre.value_counts() # animation == same number as before

drama              8098
comedy             6501
adventure          2818
science fiction    2719
western            2660
thriller           2516
action             2116
musical            1827
noir               1713
horror             1355
romance            1332
fantasy             827
animation           811
war                 534
sport               409
documentary         354
family              319
historical          253
biography           219
Name: Genre, dtype: int64

## SVM

### Binary classification (western)

In [8]:
from sklearn.model_selection import train_test_split

X = df['Plot'].values
y = df['Genre'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, shuffle=True)

In [9]:
import numpy as np 

sample_idx = 0
y_train_bin = np.asarray(y_train)==y_train[sample_idx]
y_test_bin = np.asarray(y_test)==y_train[sample_idx]
y_train_bin,y_test_bin

(array([ True, False,  True, ..., False, False, False]),
 array([False, False,  True, ...,  True, False, False]))

In [11]:
len(y_train_bin), len(y_test_bin)

(26166, 11215)

In [12]:
import nltk
import spacy

nlp = spacy.load('en_core_web_sm')

doc_counter = 0
def reset_counter():
  global doc_counter
  doc_counter = 0

def increase_counter():
  global doc_counter
  doc_counter += 1
  if doc_counter % 100 == 0:
    print(doc_counter)

def spacy_nlp_tokenizer(text):
    increase_counter()
        
    # we use spacy for main nlp tasks
    doc = nlp(text)
    # lemmatized tokens, skipping stopwords
    lemmas = ['LEMMA_'+token.lemma_ for token in doc if not token.is_stop and token.ent_type != "PERSON"]
    # entity_types
    # entity_types = ['NER_'+token.ent_type_ for token in doc if token.ent_type_]

    # in case an entity linker is available, we can use it do put actual entities as
    # features, e.g. Queen Elizabeth, Elizabeth II, Her Majesty -> KB2912
    # see https://spacy.io/usage/training#entity-linker
    # entities = ['ENT_'+token.ent_kb_id_ for token in doc if token.ent_kb_id_]

    # we use a simple nltk function to create ngrams
    lemma_bigrams = ['BI_'+p1+'_'+p2 for p1,p2 in nltk.ngrams(lemmas,2)]
    #lemma_trigrams = ['TRI_'+p1+'_'+p2+'_'+p3 for p1,p2,p3 in nltk.ngrams(lemmas,3)]

    all_tokens = list()
    all_tokens.extend(lemmas)
    all_tokens.extend(lemma_bigrams)
    #all_tokens.extend(lemma_trigrams)
    #all_tokens.extend(entity_types)
    return all_tokens

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(analyzer=spacy_nlp_tokenizer, min_df=5)

X_train_tok = vect.fit_transform(X_train)

X_test_tok = vect.transform(X_test)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'sel__k': [1000, 2000, 5000], 'learner__C': [0.01, 0.1, 1]}, 
 ]

opt_pipeline = Pipeline([
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', LinearSVC())  # learning algorithm
])


n_jobs = 4

opt_search = GridSearchCV(opt_pipeline, param_grid, cv=5, n_jobs = n_jobs, verbose=3).fit(X_train_tok,y_train_bin)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


In [15]:
opt_search.best_params_

{'learner__C': 0.1, 'sel__k': 5000}

In [17]:
opt_predictions = opt_search.best_estimator_.predict(X_test_tok)

correct = 0
for prediction,true_label in zip(opt_predictions, y_test_bin):
    if prediction==true_label:
        correct += 1
print("Accuracy:\t", correct/len(opt_predictions))

Accuracy:	 0.8507356219349086


In [19]:
from sklearn.metrics import confusion_matrix, classification_report
print('Classification report:')
print(classification_report(y_test_bin, opt_predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test_bin, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

       False       0.86      0.98      0.92      9265
        True       0.73      0.22      0.34      1950

    accuracy                           0.85     11215
   macro avg       0.79      0.60      0.63     11215
weighted avg       0.84      0.85      0.82     11215

Confusion matrix:
[[9105  160]
 [1514  436]]


### Multiclass 

In [2]:
from sklearn.model_selection import train_test_split

X = df['Plot'].values
y = df['Genre'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, shuffle=True)

In [6]:
len(X_train),len(y_train),len(X_test),len(y_test)

(26166, 26166, 11215, 11215)

In [32]:
import nltk
import spacy

nlp = spacy.load('en_core_web_sm')

doc_counter = 0
def reset_counter():
  global doc_counter
  doc_counter = 0

def increase_counter():
  global doc_counter
  doc_counter += 1
  if doc_counter % 100 == 0:
    print(doc_counter)

def spacy_nlp_tokenizer(text):
    increase_counter()
        
    # we use spacy for main nlp tasks
    doc = nlp(text)
    # lemmatized tokens, skipping stopwords
    lemmas = ['LEMMA_'+token.lemma_ for token in doc if not token.is_stop]
    # entity_types
    # entity_types = ['NER_'+token.ent_type_ for token in doc if token.ent_type_]

    # in case an entity linker is available, we can use it do put actual entities as
    # features, e.g. Queen Elizabeth, Elizabeth II, Her Majesty -> KB2912
    # see https://spacy.io/usage/training#entity-linker
    # entities = ['ENT_'+token.ent_kb_id_ for token in doc if token.ent_kb_id_]

    # we use a simple nltk function to create ngrams
    lemma_bigrams = ['BI_'+p1+'_'+p2 for p1,p2 in nltk.ngrams(lemmas,2)]
    lemma_trigrams = ['TRI_'+p1+'_'+p2+'_'+p3 for p1,p2,p3 in nltk.ngrams(lemmas,3)]

    all_tokens = list()
    all_tokens.extend(lemmas)
    all_tokens.extend(lemma_bigrams)
    all_tokens.extend(lemma_trigrams)
    #all_tokens.extend(entity_types)
    return all_tokens

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(analyzer=spacy_nlp_tokenizer, min_df=5)

X_train_tok = vect.fit_transform(X_train)

X_test_tok = vect.transform(X_test)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [34]:
X_train_tok.shape, X_test_tok.shape

((26166, 245531), (11215, 245531))

In [35]:
#writing features to pickle
import pickle

with open('x_train_tok.pkl','wb') as train_outfile:
    pickle.dump(X_train_tok,train_outfile)
with open('x_test_tok.pkl','wb') as test_outfile:
    pickle.dump(X_test_tok,test_outfile)

In [9]:
#reading from pickle
import pickle

with open('x_train_tok.pkl','rb') as train_file:
    X_train_tok = pickle.load(train_file)

with open('x_test_tok.pkl','rb') as test_file:
    X_test_tok = pickle.load(test_file)


In [10]:
X_train_tok.shape, X_test_tok.shape

((29904, 281835), (7477, 281835))

In [36]:
for feat,freq in zip(vect.inverse_transform(X_train_tok[0,:])[0],X_train_tok[0,:].data):
    print(feat,freq)

LEMMA_opening 3
LEMMA_scene 4
LEMMA_, 63
LEMMA_24 1
LEMMA_- 10
LEMMA_year 8
LEMMA_old 1
LEMMA_" 14
LEMMA_Alice 23
LEMMA_Ayres 2
LEMMA_( 7
LEMMA_Portman 1
LEMMA_) 7
LEMMA_Dan 30
LEMMA_Woolf 1
LEMMA_Law 1
LEMMA_time 2
LEMMA_opposite 1
LEMMA_side 1
LEMMA_street 3
LEMMA_walk 2
LEMMA_rush 2
LEMMA_hour 1
LEMMA_pedestrian 1
LEMMA_. 50
LEMMA_young 3
LEMMA_american 2
LEMMA_stripper 2
LEMMA_arrive 2
LEMMA_London 4
LEMMA_unsuccessful 1
LEMMA_british 2
LEMMA_writer 1
LEMMA_way 3
LEMMA_work 2
LEMMA_write 2
LEMMA_obituary 1
LEMMA_newspaper 1
LEMMA_look 1
LEMMA_wrong 1
LEMMA_direction 1
LEMMA_cross 1
LEMMA_hit 2
LEMMA_taxi 1
LEMMA_cab 1
LEMMA_right 2
LEMMA_eye 1
LEMMA_smile 1
LEMMA_say 1
LEMMA_hello 1
LEMMA_strange 1
LEMMA_take 4
LEMMA_hospital 1
LEMMA_treat 1
LEMMA_release 1
LEMMA_afterward 1
LEMMA_office 2
LEMMA_stop 1
LEMMA_Park 2
LEMMA_park 1
LEMMA_father 1
LEMMA_visit 1
LEMMA_mother 1
LEMMA_death 1
LEMMA_pause 1
LEMMA_leave 6
LEMMA_go 5
LEMMA_remind 1
LEMMA_traffic 1
LEMMA_England 1
LEMMA_tend 1

In [50]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'sel__k': [1000, 2000, 5000, 10000], 'learner__C': [0.01, 0.1, 1, 10]}, 
 ]

opt_pipeline = Pipeline([
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', LinearSVC())  # learning algorithm
])


n_jobs = 5

opt_search = GridSearchCV(opt_pipeline, param_grid, cv=5, n_jobs = n_jobs, verbose=3).fit(X_train_tok,y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


In [51]:
opt_search.best_params_

{'learner__C': 0.1, 'learner__class_weight': None, 'sel__k': 10000}

In [52]:
opt_predictions = opt_search.best_estimator_.predict(X_test_tok)

correct = 0
for prediction,true_label in zip(opt_predictions, y_test):
    if prediction==true_label:
        correct += 1
print("Accuracy:\t", correct/len(opt_predictions))

Accuracy:	 0.5410610789121711


In [54]:
from sklearn.metrics import confusion_matrix, classification_report
print('Classification report:')
print(classification_report(y_test, opt_predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test, opt_predictions)
print(cm)

Classification report:
                 precision    recall  f1-score   support

         action       0.48      0.39      0.43       635
      adventure       0.58      0.63      0.60       845
      animation       0.72      0.40      0.51       243
      biography       0.00      0.00      0.00        66
         comedy       0.48      0.60      0.53      1950
    documentary       0.84      0.30      0.44       106
          drama       0.46      0.65      0.54      2430
         family       1.00      0.02      0.04        96
        fantasy       0.34      0.10      0.16       248
     historical       0.00      0.00      0.00        76
         horror       0.66      0.47      0.55       406
        musical       0.65      0.46      0.54       548
           noir       0.68      0.46      0.55       514
        romance       0.35      0.15      0.21       400
science fiction       0.68      0.80      0.74       816
          sport       0.64      0.20      0.31       123
       

C:\Users\Jacopo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jacopo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jacopo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Feature inspection

In [58]:
tokenizer = vect
selector = opt_search.best_estimator_.named_steps['sel']
classifier = opt_search.best_estimator_.named_steps['learner']
feature_names = tokenizer.get_feature_names()
feats_w_score = list()
for index,(selected,score) in enumerate(zip(selector.get_support(),selector.scores_)):
    feats_w_score.append((score,selected,feature_names[index]))
feats_w_score = sorted(feats_w_score)
print("BEST FEATURES")
feats_w_score[:-100:-1]

BEST FEATURES


[(31960.26061009445, True, 'LEMMA_,'),
 (25169.719175891325, True, 'LEMMA_Jerry'),
 (23497.87765729788, True, 'LEMMA_Tom'),
 (19414.526498037154, True, 'LEMMA_alien'),
 (18255.576534556116, True, 'LEMMA_Bugs'),
 (17365.15539605599, True, 'LEMMA_.'),
 (14910.472293567726, True, 'LEMMA_Earth'),
 (12722.514849501827, True, 'LEMMA_kill'),
 (12543.152548994987, True, 'LEMMA_\r\n'),
 (12275.379976263155, True, 'LEMMA_bug'),
 (12209.715375859047, True, 'BI_LEMMA_._LEMMA_\r\n'),
 (10449.587459661154, True, 'LEMMA_ranch'),
 (9877.591296937513, True, 'LEMMA_ '),
 (9364.7895934113, True, 'LEMMA_human'),
 (9108.79876921604, True, 'LEMMA_german'),
 (8977.904918759263, True, 'LEMMA_"'),
 (8816.65874829676, True, 'LEMMA_planet'),
 (8599.44459234337, True, 'LEMMA_Spike'),
 (8553.933299941911, True, 'BI_LEMMA_(_LEMMA_"'),
 (8547.54399197894, True, 'BI_LEMMA_._LEMMA_Tom'),
 (8482.497431599035, True, 'BI_LEMMA_._LEMMA_ '),
 (7778.801480808113, True, 'LEMMA_creature'),
 (7619.040317488679, True, 'LEMMA_ca

In [130]:
feats_w_classifier_weight = dict()

for n, genre in enumerate(sorted(set(y_test))):
    feats_w_classifier_weight[genre] = list()
    for index,weight in enumerate(selector.inverse_transform(classifier.coef_)[n]):
        if weight!=0:
            feats_w_classifier_weight[genre].append((weight,feature_names[index]))
    

In [133]:
sorted(feats_w_classifier_weight['thriller'])[:-100:-1]

[(1.2342882449030586, 'LEMMA_murder'),
 (1.0795252640393822, 'LEMMA_suspect'),
 (0.9297482045996174, 'BI_LEMMA_._LEMMA_\r\n'),
 (0.8693167360898718, 'LEMMA_police'),
 (0.8531953052531958, 'LEMMA_mystery'),
 (0.7743584620060625, 'LEMMA_phone'),
 (0.7577972040921012, 'LEMMA_blackmailer'),
 (0.7484058024360729, 'LEMMA_Holmes'),
 (0.722077196362928, 'LEMMA_\r\n'),
 (0.6729092062093767, 'LEMMA_murderer'),
 (0.6347309762915365, 'LEMMA_killer'),
 (0.6268576555082526, 'LEMMA_Poirot'),
 (0.6229262446723992, 'LEMMA_London'),
 (0.6129177131676552, 'LEMMA_detective'),
 (0.6039835684543726, 'LEMMA_Chan'),
 (0.600337250705263, 'LEMMA_Watson'),
 (0.5991711505365434, 'LEMMA_criminal'),
 (0.5871564519953145, 'LEMMA_flat'),
 (0.5858822794309061, 'BI_LEMMA_dead_LEMMA_.'),
 (0.5818362699519485, 'BI_LEMMA_prime_LEMMA_suspect'),
 (0.5657130773259175, 'BI_LEMMA_find_LEMMA_dead'),
 (0.5536142653399997, 'LEMMA_Matty'),
 (0.5483794093207841, 'LEMMA_dead'),
 (0.540083073727887, 'LEMMA_Police'),
 (0.5371588626769

In [91]:
for n, genre in enumerate(set(y_test)):
    print(n, genre)

0 science fiction
1 romance
2 horror
3 adventure
4 action
5 biography
6 musical
7 drama
8 thriller
9 noir
10 historical
11 fantasy
12 animation
13 war
14 sport
15 documentary
16 western
17 comedy
18 family
